In [56]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date, ForeignKey, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship
from datetime import datetime, timedelta
import json
import re
import numpy as np
from functools import reduce
import requests
from api_keys import email, passw #Please add to .gitnore file your own individual usernames and passwords. Also set up your account on the website with a watched area of your desire.

In [57]:
browser = Browser('chrome')
url = "https://housesigma.com/listings/watched-area-and-community"
browser.visit(url)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [58]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)
email = email
passw = passw
login = (email) 
email_input.send_keys(login)

In [59]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [60]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()

In [61]:
browser.is_element_present_by_css('.pc-listing-card', wait_time=10)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [64]:
data_list = []  

articles = soup_obj.find_all('article', class_='pc-listing-card')

for article in articles:
    data = {}

    # Address
    address_element = article.find('h3', class_='address')
    data['Address'] = address_element.text.strip() if address_element else None

    # Neighbourhood
    text1_element = article.find('span', class_='text1')
    data['Neighbourhood'] = text1_element.text.strip() if text1_element else None

    json_scripts = article.find_all('script', class_='hs-script-home-struct', type='application/ld+json')


    for script in json_scripts:
        json_data = script.string
        if json_data:
            json_dict = json.loads(json_data)
            
            # Long and Lat and floor size
            if 'floorSize' in json_dict:
                data['Floor Size'] = json_dict['floorSize']['value']
            if 'geo' in json_dict:
                data['Latitude'] = json_dict['geo']['latitude']
                data['Longitude'] = json_dict['geo']['longitude']

    # Date 
    date_preview_element = article.find('div', class_='date-preview')
    data['Date of Status'] = date_preview_element.text.strip() if date_preview_element else None

    # Price
    highlight_element = article.find('span', class_='highlight')
    if highlight_element:
        data['Price Listed'] = highlight_element.text.strip()
    else:
        line_through_element = article.find('span', class_='line-through')
        data['Price Listed'] = line_through_element.text.strip() if line_through_element else None
   
    # Status
    status_element = article.select_one('div[class^="status-type"]')
    data['Status'] = status_element.text.strip() if status_element else None

    # Sold Price (if status is "Sold" or "Sold Conditional")
    if data['Status'] in ["Sold", "Sold Conditional"]:
        sold_price_element_special = article.select_one('div.price-area span.special')
        sold_price_element_highlight_special = article.select_one('div.price-area span.highlight.special')

        if sold_price_element_special:
            data['Sold Price'] = sold_price_element_special.text.strip()
        elif sold_price_element_highlight_special:
            data['Sold Price'] = sold_price_element_highlight_special.text.strip()
        else:
            data['Sold Price'] = None
    else:
        data['Sold Price'] = 0
    
    # Type of house
    type_element = article.find('p', class_='type')
    data['Type of House'] = type_element.text.strip() if type_element else None


    #Bathroom
    if re.search(r'\b(?:bathroom)\b', text):
        bathroom_info = re.findall(r'\d+', text)
        if bathroom_info:
            data['Bathrooms'] = int(bathroom_info[0]) 

    # Bedroom
    if re.search(r'\b(?:bedroom)\b', text):
        bedroom_info = re.findall(r'\d+', text)
        if bedroom_info:
            data['Bedrooms'] = int(bedroom_info[0]) 
        
    # Garage
    if re.search(r'\b(?:garage)\b', text):
        garage_info = re.findall(r'\d+', text)
        if garage_info:
            data['Garage'] = int(garage_info[0]) 
                
    data_list.append(data)

for i, data in enumerate(data_list, 1):
    print(f"{i}: {data}")


1: {'Address': '1307 - 297 Oak Walk Drive , Oakville - Uptown Core', 'Neighbourhood': 'Uptown Core', 'Floor Size': 655, 'Latitude': -79.718298, 'Longitude': 43.486047, 'Date of Status': '59 minutes ago', 'Price Listed': '$625,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo/Apt Unit', 'Garage': 1}
2: {'Address': '2048 Ardleigh Road , Oakville - Eastlake', 'Neighbourhood': 'Eastlake', 'Floor Size': 4500, 'Latitude': -79.644237, 'Longitude': 43.472015, 'Date of Status': '1 hour ago', 'Price Listed': '$7,499,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Single Family Residence', 'Garage': 1}
3: {'Address': '306 - 390 Upper Middle Road E , Oakville - College Park', 'Neighbourhood': 'College Park', 'Floor Size': 1900, 'Latitude': -79.699995, 'Longitude': 43.476247, 'Date of Status': '1 hour ago', 'Price Listed': '$1,499,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo/Apt Unit', 'Garage': 1}
4: {'Address': '107 - 269 Georgian Dr , Oak

In [65]:
def convert_relative_timestamp(relative_timestamp):
    if " hours ago" in relative_timestamp:
        hours = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(hours=hours)
        return absolute_timestamp.date()
    elif " days ago" in relative_timestamp:
        days = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(days=days)
        return absolute_timestamp.date()
    else:
        return datetime.now().date()


df = pd.DataFrame(data_list)
df['City'] = df['Address'].str.split(',').str[1].str.split(' - ').str[0]
df['Address'] = df['Address'].str.split(',').str[0]
df['Date of Status'] = df['Date of Status'].apply(convert_relative_timestamp)

df['Sold Price'] = df['Sold Price'].str.replace('$', '').str.replace(',', '').astype(float)
df['Price Listed'] = df['Price Listed'].str.replace('$', '').str.replace(',', '').astype(float)
df

,Address,Neighbourhood,Floor Size,Latitude,Longitude,Date of Status,Price Listed,Status,Sold Price,Type of House,Garage,City
0,1307 - 297 Oak Walk Drive,Uptown Core,655.0,-79.718298,43.486047,2024-02-05,625000.0,For Sale,NaN,Condo/Apt Unit,1,Oakville
1,2048 Ardleigh Road,Eastlake,4500.0,-79.644237,43.472015,2024-02-05,7499000.0,For Sale,NaN,Single Family Residence,1,Oakville
2,306 - 390 Upper Middle Road E,College Park,1900.0,-79.699995,43.476247,2024-02-05,1499000.0,For Sale,NaN,Condo/Apt Unit,1,Oakville
3,107 - 269 Georgian Dr,Uptown Core,1099.0,-79.715680,43.481330,2024-02-05,749900.0,For Sale,NaN,Condo Townhouse,1,Oakville
4,7 - 2300 Brays Lane,Glen Abbey,1299.0,-79.738670,43.426420,2024-02-05,809888.0,Sold Conditional,809888.0,Condo Townhouse,1,Oakville
5,339 - 2485 Taunton Rd,Uptown Core,849.0,-79.719380,43.486300,2024-02-05,699000.0,For Sale,NaN,Condo Apt,1,Oakville
6,18 - 1120 Queens Ave,College Park,1899.0,-79.690600,43.462660,2024-02-05,948000.0,Sold,940000.0,Condo Townhouse,1,Oakville
7,14 - 2295 Rochester Circ,Palermo West,2250.0,-79.765810,43.426190,2024-02-05,1349900.0,For Sale,NaN,Freehold Townhouse,1,Oakville
8,382 Queen Mary Dr,Old Oakville,NaN,-79.682680,43.448050,2024-02-05,1149000.0,For Sale,NaN,Detached,1,Oakville
9,352 Maple Grove Dr,Eastlake,4250.0,-79.652370,43.475070,2024-02-05,4098000.0,Terminated,NaN,Detached,1,Oakville


In [66]:
weather_data_oakville = 'Resources/Oakville_Historical_Weather.csv'
selected_columns = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE','LOCAL_DATE', 'STATION_NAME', 'COOLING_DEGREE_DAYS', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_YEAR', 'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE', 'ID']
df_weather = pd.read_csv(weather_data_oakville, usecols=selected_columns, encoding="ISO-8859-1", on_bad_lines='skip', low_memory=True, engine='python')
df_weather = df_weather.dropna(subset=['MIN_TEMPERATURE'])
df_weather = df_weather.reset_index(drop=True)
columns_to_fill = ['HEATING_DEGREE_DAYS', 'MIN_TEMPERATURE', 'COOLING_DEGREE_DAYS', 
                   'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'MEAN_TEMPERATURE', 
                   'TOTAL_SNOW', 'TOTAL_RAIN', 'MAX_TEMPERATURE']
df_weather[columns_to_fill] = df_weather[columns_to_fill].fillna(0)
df_weather['LOCAL_DATE'] = pd.to_datetime(df_weather['LOCAL_DATE']).dt.date
df_weather.sample(10)

,HEATING_DEGREE_DAYS,MIN_TEMPERATURE,LOCAL_DATE,STATION_NAME,COOLING_DEGREE_DAYS,LOCAL_MONTH,LOCAL_DAY,LOCAL_YEAR,TOTAL_PRECIPITATION,SNOW_ON_GROUND,MEAN_TEMPERATURE,TOTAL_SNOW,TOTAL_RAIN,MAX_TEMPERATURE,ID
5565,6.4,5.3,2005-11-13,OAKVILLE GERARD,0.0,11,13,2005,0.0,0.0,11.6,0.0,0.0,17.8,6155PD4.2005.11.13
1482,0.0,17.0,1994-06-14,OAKVILLE GERARD,2.5,6,14,1994,0.0,0.0,20.5,0.0,0.0,24.0,6155PD4.1994.6.14
3538,21.5,-4.5,2000-02-14,OAKVILLE GERARD,0.0,2,14,2000,6.0,29.0,-3.5,6.0,0.0,-2.5,6155PD4.2000.2.14
2226,0.0,18.0,1996-07-06,OAKVILLE GERARD,4.8,7,6,1996,0.0,0.0,22.8,0.0,0.0,27.5,6155PD4.1996.7.6
4721,0.0,18.5,2003-06-27,OAKVILLE GERARD,4.8,6,27,2003,0.0,0.0,22.8,0.0,0.0,27.0,6155PD4.2003.6.27
291,12.5,2.5,1991-02-05,OAKVILLE GERARD,0.0,2,5,1991,0.0,0.0,5.5,0.0,0.0,8.5,6155PD4.1991.2.5
2016,22.8,-10.0,1995-12-08,OAKVILLE GERARD,0.0,12,8,1995,11.0,0.0,-4.8,11.0,0.0,0.5,6155PD4.1995.12.8
4707,2.2,10.5,2003-06-13,OAKVILLE GERARD,0.0,6,13,2003,0.0,0.0,15.8,0.0,0.0,21.0,6155PD4.2003.6.13
1852,0.0,20.0,1995-06-26,OAKVILLE GERARD,5.3,6,26,1995,0.6,0.0,23.3,0.0,0.6,26.5,6155PD4.1995.6.26
4390,0.0,19.0,2002-07-31,OAKVILLE GERARD,7.8,7,31,2002,1.2,0.0,25.8,0.0,1.2,32.5,6155PD4.2002.7.31


In [67]:
Base = declarative_base()
#Base.metadata.clear()

C:\Users\qwert\AppData\Local\Temp\ipykernel_3232\2374365428.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [68]:
class House(Base):
    __tablename__ = 'houses'
    id = Column(Integer, primary_key=True)
    address = Column(String)
    status = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    floor_size = Column(Integer)
    bedrooms = Column(Integer)  
    bathrooms = Column(Integer) 
    garage = Column(Integer)
    city = Column(String)
    type_of_house = Column(String)  
    date_listed = Column(Date)
    neighbourhood = Column(String)
    price = Column(Float)

class Sold(Base):
    __tablename__ = 'sold'
    id = Column(Integer, primary_key=True)
    address = Column(String, ForeignKey('houses.address'))
    price = Column(Float)
    status = Column(String)
    date_listed = Column(Date)
    sold_price = Column(Float)
    type_of_house = Column(String)  
    city = Column(String)
    neighbourhood = Column(String)

class WeatherData(Base):
    __tablename__ = 'weather_data'

    id = Column(String, primary_key=True, unique=True) 
    heating_degree_days = Column(Float)
    min_temperature = Column(Float)
    local_date = Column(String)
    station_name = Column(String)
    cooling_degree_days = Column(Float)
    local_month = Column(Integer)
    local_day = Column(Integer)
    local_year = Column(Integer)
    total_precipitation = Column(Float)
    snow_on_ground = Column(Float)
    mean_temperature = Column(Float)
    total_snow = Column(Float)
    total_rain = Column(Float)
    max_temperature = Column(Float)

In [69]:
engine = create_engine('sqlite:///housing_sales_db.db')
session = Session(bind=engine)
#Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
session.commit()
session.close()

In [70]:
engine = create_engine('sqlite:///housing_sales_db.db')
session = Session(bind=engine)

In [71]:
for data in data_list:
    address = data.get('Address')
    if address:
        house_exists = session.query(House).filter_by(address=address).first()

    if house_exists:
        house_exists.status = data.get('Status')
        house_exists.date_listed = convert_relative_timestamp(data.get('Date of Status'))
    else:
        address_parts = address.split(',')
        city = address_parts[1].split(' - ')[0].strip()
        address = address_parts[0]

        new_house = House(
            address=address,
            status=data.get('Status'),
            latitude=data.get('Latitude'),
            longitude=data.get('Longitude'),
            floor_size=data.get('Floor Size'),
            bedrooms=data.get('Bedrooms'),
            bathrooms=data.get('Bathrooms'),
            garage=data.get('Garage'),
            city=city,  
            type_of_house=data.get('Type of House'),
            date_listed=convert_relative_timestamp(data.get('Date of Status')),
            neighbourhood=data.get('Neighbourhood'),
            price=float(data.get('Price Listed').replace('$', '').replace(',', ''))
        )
        session.add(new_house)

In [72]:
for data in data_list:
    address = data.get('Address')
    status = data.get('Status')

    if status in ["Sold", "Sold Conditional"]:
        sold_house = session.query(Sold).filter_by(address=address).first()

        if sold_house:
            sold_house.status = status
            sold_house.date_listed = convert_relative_timestamp(data.get('Date of Status'))
            sold_house.sold_price = float(str(data.get('Sold Price')).replace('$', '').replace(',', ''))
        else:
            address_parts = address.split(',')
            city = address_parts[1].split(' - ')[0].strip()
            address = address_parts[0]
            new_sold_house = Sold(
                address=address,
                price=float(data.get('Price Listed').replace('$', '').replace(',', '')),
                status=status,
                date_listed=convert_relative_timestamp(data.get('Date of Status')),
                sold_price=float(str(data.get('Sold Price')).replace('$', '').replace(',', '')),
                city=city,
                type_of_house=data.get('Type of House'),
                neighbourhood=data.get('Neighbourhood')
            )
            session.add(new_sold_house)

In [73]:
weather_data_instances = []

for index, row in df_weather.iterrows():
    weather_data_instance = WeatherData(
        id=row['ID'],
        heating_degree_days=row['HEATING_DEGREE_DAYS'],
        min_temperature=row['MIN_TEMPERATURE'],
        local_date=row['LOCAL_DATE'],
        station_name=row['STATION_NAME'],
        cooling_degree_days=row['COOLING_DEGREE_DAYS'],
        local_month=row['LOCAL_MONTH'],
        local_day=row['LOCAL_DAY'],
        local_year=row['LOCAL_YEAR'],
        total_precipitation=row['TOTAL_PRECIPITATION'],
        snow_on_ground=row['SNOW_ON_GROUND'],
        mean_temperature=row['MEAN_TEMPERATURE'],
        total_snow=row['TOTAL_SNOW'],
        total_rain=row['TOTAL_RAIN'],
        max_temperature=row['MAX_TEMPERATURE']
    )
    weather_data_instances.append(weather_data_instance)

session.add_all(weather_data_instances)

In [74]:
session.commit()

In [75]:
sold_count = session.query(func.count(Sold.id)).scalar()
house_count = session.query(func.count(House.id)).scalar()
weather_count = session.query(func.count(WeatherData.id)).scalar()
print("Number of records in Weather table:", weather_count)
print("Number of records in Sold table:", sold_count)
print("Number of records in House table:", house_count)

Number of records in Weather table: 5894
Number of records in Sold table: 2
Number of records in House table: 10


In [76]:
session.close()

In [77]:
browser.quit()